In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install pandas numpy

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.0 MB 1.5 MB/s eta 0:00:07
   - -------------------------------------- 0.5/11.0 MB 1.5 MB/s eta 0:00:07
   --- ------------------------------------ 1.0/11.0 MB 1.1 MB/s eta 0:00:09
   ---- ----------------------------------- 1.3/11.0 MB 1.2 MB/s eta 0:00:09
   ----- ---------------------------------- 1.6/11.0 MB 1.2 MB/s eta 0:00:08
   ----- ---------------------------------- 1.6/11.0 MB 1.2 MB/s eta 0:00:08
   ------- -------------------------------- 2.1/11.0 MB 1.2 MB/s eta 0:00:08
   -------- ------------------------------- 2.4/11.0 MB 1.2 MB/s eta 0:00:07
   --------- ------------------------------ 2.6/11.0 MB 1.2 MB/s eta 0:00:07
   ---------- ----------------------------- 2.9/11.0 MB 1.3 MB/s eta 0:00:07
   ---------- ------

In [ ]:
!pip install pynini

In [3]:
!pip install graphviz 


In [3]:
#importation des librairies necessaires 
import pynini 
from pynini.lib import pynutil


In [4]:

def apply_fst(text, fst):
    try:
        return(pynini.shortestpath(pynini.accep(text,token_type='utf8') @ fst).string("utf8"))

    except Exception as e:
        return(f"Error: {e}, for input:'{text}'")

In [5]:
# Cell 3: Helper function to create input:output FSTs
# This function creates an FST that transduces a single input string to a single output string.


def I_O_FST(input_str: str, output_str: str) -> pynini.Fst:
    """Creates an FST mapping input_str to output_str."""

    # Sécuriser les entrées
    input_str = str(input_str)
    output_str = str(output_str)

    # Créer les acceptors
    input_accep = pynini.accep(input_str, token_type="utf8")
    output_accep = pynini.accep(output_str, token_type="utf8")

    # Créer le transducteur input → output
    fst = pynini.cross(input_accep, output_accep)

    return fst.optimize()


In [6]:
# Cell 4: Define FSTs for Units (0-9)
#The I_O_FST function creates an FST mapping from one grammar to the other
#I_O_FST(input_str: str, output_str: str)
digit_map = {
    "0": "zéro",
    "1": "un",
    "2": "deux",
    "3": "trois",
    "4": "quatre",
    "5": "cinq",
    "6": "six",
    "7": "sept",
    "8": "huit",
    "9": "neuf",
}
################CODE STARTING FROM HERE#####################################
# fill in the your code in the list comprehension below it should map from the units_map.keys() to units_map.values(), i.e: "0": "zero", as a result of iterating through units_map.items()
fst_units_list = [I_O_FST(k, v) for k, v  in digit_map.items()]
################CODE ENDS HERE##############################################
fst_units_base = pynini.union(*fst_units_list).optimize()

# --- NOUVEAU CODE POUR GÉRER LES ZÉROS NON SIGNIFICATIFS ---

# FST qui supprime un seul zéro d'entrée (0:epsilon)
# pynutil.delete("0") crée un FST qui lit "0" et sort la chaîne vide.
delete_zero = pynutil.delete("0") 

# FST qui gère UN ou PLUSIEURS zéros non significatifs (0+ -> epsilon)
# .plus() dans pynini équivaut à la fermeture de Kleene positive (1 ou plus)
fst_leading_zeros = delete_zero.plus 

# Rendre cette séquence de zéros facultative (0* -> epsilon)
# .closure() dans pynini équivaut à la fermeture de Kleene (0 ou plus)
# ATTENTION: Si on fait cela, ça permettrait aussi "1", "2" etc. sans avoir besoin de la ligne delete_zero.plus()
# Pour être précis et plus efficace, on va utiliser pynini.closure() sur le zéro simple, puis rendre cette séquence facultative AVANT le nombre réel.
# C'est plus simple de dire : 'Zéro facultatif (0 ou plus) sur l'entrée, suivi du FST de base.'

# FST qui supprime ZERO ou PLUSIEURS zéros d'entrée (0* -> epsilon)
# delete_zero.closure() est incorrect car delete_zero est 0:epsilon, et sa closure permettrait d'ignorer 0, 00, 000, etc.
# L'approche la plus courante est de rendre l'occurrence de la séquence 0+ facultative:
fst_optional_leading_zeros = fst_leading_zeros.optimize().closure().optimize()

# La meilleure façon pour le premier chiffre est de :
# 1. Accepter 0* -> epsilon (ignorer les zéros)
# 2. Suivi par le reste du nombre.

# Le cas spécial est le nombre "0" lui-même, qui est géré par fst_units_base ("0": "zéro").

# FST qui supprime un zéro ou plus (0+ -> epsilon) rendu facultatif (0* -> epsilon)
# On utilise pynutil.closure(delete_zero) pour accepter 0 ou plus zéros d'entrée sans sortie.
fst_zero_prefix = delete_zero.star

# 2. Concaténer
# Le FST final est : (Zéros non significatifs facultatifs) CONCATÉNÉ AVEC (Conversion unité)
fst_units_with_leading_zeros = (fst_zero_prefix + fst_units_base).optimize()

In [7]:
apply_fst("09",fst_units_with_leading_zeros)

'neuf'

In [8]:
# Cell 5: Define FSTs for Teens (10-19) - These are irregular

teens_map = {
    "10": "dix",
    "11": "onze",
    "12": "douze",
    "13": "treize",
    "14": "quatorze",
    "15": "quinze",
    "16": "seize",
    "17": "dix-sept",
    "18": "dix-huit",
    "19": "dix-neuf",
}
################CODE STARTING FROM HERE#####################################

fst_teens_list = [I_O_FST(k, v) for k, v in teens_map.items()]

# pynini.union takes multiple finite-state transducers (FSTs) or acceptors as input and creates a new FST that accepts any string accepted by at least one of its input FSTs
# fst_teens_list is a segmented list of FSTs, where each individual cardinal number maps to each individual english word, i.e: fst_teens_list[0] = "10" to "ten", fst_teens_list[0] = "11" to "eleven"
# use the * operator to unpack values from fst_units_list, so that pynini.union receives each FST as a separate argument.
fst_teens = pynini.union(*fst_teens_list).optimize()
################CODE ENDS HERE##############################################

# Test (optional)
# print(pynini.shortestpath(pynini.accep("15", token_type=sigma_table) @ fst_teens).string(sigma_table))

In [9]:
apply_fst("11",fst_teens)

'onze'

In [10]:
# Cell 6: Define FSTs for Tens Prefixes (for 20, 30, 40) and the "0" part

# These map the first digit of a "tens" number to its prefix
# e.g., "2" -> "twenty", "3" -> "thirty"
tens_digit_map ={
    "2": "vingt",
    "3": "trente",
    "4": "quarante",
    "5": "cinquante",
    "6": "soixante",
}
fst_tens_digit_list = [I_O_FST(digit, text) for digit, text in tens_digit_map.items()]
fst_tens_digits = pynini.union(*fst_tens_digit_list).optimize()
# FST to handle the "0" in "20", "30", "40" (consumes "0", outputs nothing)
fst_eat_zero = I_O_FST("0", "") # "0" -> <eps>
# FST for exact tens (20, 30, 40)
# This concatenates the tens_digit FST with the eat_zero FST
# e.g., ("2" : "twenty") + ("0" : "")  applied to "20" yields "twenty"
# Assume these come from your FSTs

## When you write fst1 + fst2 in Pynini, it means concatenation of two FSTs.
## creating a new FST that accepts any string consisting of a string recognized by fst1 directly followed by a string recognized by fst2.
fst_exact_tens = (fst_tens_digits + fst_eat_zero).optimize()

fst_exact_tens.optimize()

ERROR:root:Dot rendering failed: [WinError 2] Le fichier spécifié est introuvable


<vector Fst at 0x1bf49099c30>

In [11]:
apply_fst("60", fst_exact_tens)

'soixante'

In [12]:
# Cell 7: Define FSTs for Compound Tens (21-29, 31-39)

# We need units for the second digit (1-9, as "0" is handled by exact_tens)
# Re-use units_map, but exclude "0"
# As a reminder, units_map = {"0": "zero", "1": "one", "2": "two", "3": "three", "4": "four","5": "five", "6": "six", "7": "seven", "8": "eight", "9": "nine"}

compound_units_map = {k: v for k, v in digit_map.items() if (k != "0" and k!="1")}
fst_compound_units_digit_list = [I_O_FST(num, text) for num, text in compound_units_map.items()]
fst_compound_units_digits = pynini.union(*fst_compound_units_digit_list).optimize()

# FST to insert a space: consumes nothing (<eps>), outputs " "
fst_insert_space = I_O_FST("", "-") # <eps> -> " "



################CODE STARTING FROM HERE#####################################
# Compound tens FST:
# (fst_tens_digits) + (fst_insert_space) + (fst_compound_units_digits)
# Example for "21":
# 1. fst_tens_digits processes "2" -> "twenty"
# 2. fst_insert_space processes <eps> -> " " (inserts space without consuming input char)
# 3. fst_compound_units_digits processes "1" -> "one"
# Result: "21" -> "twenty one"

fst_compound_tens_standard = (fst_tens_digits + fst_insert_space + fst_compound_units_digits).optimize()

# FST qui gère uniquement l'unité "1" -> "un"
fst_one_unit = I_O_FST("1", "un").optimize()
# Prenons la forme la plus propre : "vingt" + "-et-" + "un"
fst_insert_et_space = I_O_FST("", "-et-") # <eps> -> " et "

# (Dizaine) + (-et-) + (Unité 1)
fst_compound_et_un = (fst_tens_digits + fst_insert_et_space + fst_one_unit).optimize()


# 3. Union des deux chemins pour former le FST des dizaines composées
fst_compound_tens = pynini.union(fst_compound_tens_standard, fst_compound_et_un).optimize()

In [13]:
apply_fst("61", fst_compound_tens)

'soixante-et-un'

In [14]:
# Cell 8: Combine all FSTs into a single Normalizer FST

# The order of union can sometimes matter if there are ambiguities,
# but here the input domains are mostly distinct lengths or patterns.
# Pynini's union and optimize will handle it.

################CODE STARTING FROM HERE#####################################
# Using the union property combine all the FSTs that we created so far to create an FST that accepts any string accepted by at least one of its input FSTs
fst_00_to_69 = pynini.union(
    fst_units_with_leading_zeros,           # Handles "0"-"9"
    fst_teens,           # Handles "10"-"19"
    fst_exact_tens,      # Handles "20", "30", "40"
    fst_compound_tens    # Handles "21-29", "31-39"
).optimize()
################CODE ENDS HERE##############################################




In [15]:
print("Testing number normalization from 0 to 45 (expect errors > 40):\n")
for i in range(69): # Test up to 45 to see failures
    num_as_str = str(i)
    normalized_text = apply_fst(num_as_str, fst_00_to_69)
    print(f"{num_as_str:>2s} -> {normalized_text}")

Testing number normalization from 0 to 45 (expect errors > 40):

 0 -> zéro
 1 -> un
 2 -> deux
 3 -> trois
 4 -> quatre
 5 -> cinq
 6 -> six
 7 -> sept
 8 -> huit
 9 -> neuf
10 -> dix
11 -> onze
12 -> douze
13 -> treize
14 -> quatorze
15 -> quinze
16 -> seize
17 -> dix-sept
18 -> dix-huit
19 -> dix-neuf
20 -> vingt
21 -> vingt-et-un
22 -> vingt-deux
23 -> vingt-trois
24 -> vingt-quatre
25 -> vingt-cinq
26 -> vingt-six
27 -> vingt-sept
28 -> vingt-huit
29 -> vingt-neuf
30 -> trente
31 -> trente-et-un
32 -> trente-deux
33 -> trente-trois
34 -> trente-quatre
35 -> trente-cinq
36 -> trente-six
37 -> trente-sept
38 -> trente-huit
39 -> trente-neuf
40 -> quarante
41 -> quarante-et-un
42 -> quarante-deux
43 -> quarante-trois
44 -> quarante-quatre
45 -> quarante-cinq
46 -> quarante-six
47 -> quarante-sept
48 -> quarante-huit
49 -> quarante-neuf
50 -> cinquante
51 -> cinquante-et-un
52 -> cinquante-deux
53 -> cinquante-trois
54 -> cinquante-quatre
55 -> cinquante-cinq
56 -> cinquante-six
57 ->

In [16]:
# FSTs pour les nombres 10 à 19 (nécessaires pour 70-79 et 90-99)
tens_10_to_19_map = {
    "10": "dix", "11": "onze", "12": "douze", "13": "treize", "14": "quatorze",
    "15": "quinze", "16": "seize", "17": "dix-sept", "18": "dix-huit", "19": "dix-neuf"
}
fst_tens_10_to_19 = pynini.union(*[I_O_FST(k, v) for k, v in tens_10_to_19_map.items()]).optimize()

# FSTs des dizaines simples
fst_six_zero = I_O_FST("60", "soixante").optimize()
fst_quatre_vingt = I_O_FST("80", "quatre-vingts").optimize()

# FST pour insérer " et " (utilisé pour 71, 81, 91)
# fst_insert_et = I_O_FST("", " et ").optimize()
# FST pour insérer le trait d'union
fst_insert_hyphen = I_O_FST("", "-").optimize()

In [17]:
# A. Soixante-dix (70) : "6" + "10"
fst_70 = I_O_FST("7", "soixante-").optimize() + I_O_FST("0", "dix").optimize()

# B. Soixante-et-onze (71) : Règle spéciale "et-onze"
fst_71 = I_O_FST("7", "soixante").optimize() + fst_insert_et_space + I_O_FST("1", "onze").optimize()

# C. Soixante-douze à Soixante-dix-neuf (72-79) : "soixante-" + [12 à 19]
fst_tens_12_to_19 = pynini.union(*[I_O_FST(k[1], v) for k, v in tens_10_to_19_map.items() if k not in ["10", "11"]]).optimize()
fst_72_to_79 = I_O_FST("7", "soixante-").optimize() + fst_tens_12_to_19.optimize()

# Union pour 70-79
fst_70_to_79 = pynini.union(fst_70, fst_71, fst_72_to_79).optimize()

In [18]:
apply_fst("71", fst_70_to_79)

'soixante-et-onze'

In [19]:
# Redéfinition rapide des unités 1 à 9 avec séparateurs
# units_map = {"1": "un", "2": "deux", "3": "trois", "4": "quatre", "5": "cinq", "6": "six", "7": "sept", "8": "huit", "9": "neuf"}
fst_units_2_to_9 = pynini.union(*[I_O_FST(k, "-" + v) for k, v in digit_map.items() if k != "1"]).optimize()

# A. Quatre-vingts (80)
# Le 's' est la complication majeure de 80.
fst_80 = I_O_FST("8", "quatre-vingt").optimize() + I_O_FST("0", "s").optimize()

fst_insert_space =I_O_FST("", "-") # <eps> -> " - "

# B. Quatre-vingt-et-un (81)
fst_81 = I_O_FST("8", "quatre-vingt").optimize() + fst_insert_space + I_O_FST("1", "un").optimize()

# C. Quatre-vingt-deux à Quatre-vingt-neuf (82-89)
# Note: Le 's' est supprimé après 80.
fst_82_to_89 = I_O_FST("8", "quatre-vingt").optimize() + fst_units_2_to_9.optimize()

# Union pour 80-89
fst_80_to_89 = pynini.union(fst_80, fst_81, fst_82_to_89).optimize()

In [20]:
apply_fst("86", fst_80_to_89)

'quatre-vingt-six'

In [21]:
# Le FST pour la partie "quatre-vingt" sans le 's'
fst_quatre_vingt_prefix = I_O_FST("9", "quatre-vingt").optimize()

# A. Quatre-vingt-dix (90)
fst_90 = fst_quatre_vingt_prefix + I_O_FST("0", "-dix").optimize()

# B. Quatre-vingt-onze à Quatre-vingt-dix-neuf (91-99)
# On réutilise fst_tens_10_to_19 pour 11 à 19.
# Notez qu'il n'y a pas de "et" pour 91.
# On crée le FST pour 11 à 19 avec les tirets corrects pour l'ajout à "quatre-vingt"
fst_91_to_99_suffix = pynini.union(*[I_O_FST(k[1], "-" + v) for k, v in tens_10_to_19_map.items() if k != "10"]).optimize()

fst_91_to_99 = fst_quatre_vingt_prefix + fst_91_to_99_suffix

# Union pour 90-99
fst_90_to_99 = pynini.union(fst_90, fst_91_to_99).optimize()

In [22]:
apply_fst("91", fst_90_to_99)

'quatre-vingt-onze'

In [23]:
fst_70_to_99 = pynini.union(fst_70_to_79, fst_80_to_89, fst_90_to_99).optimize()

In [24]:
# Cell 8: Combine all FSTs into a single Normalizer FST

# The order of union can sometimes matter if there are ambiguities,
# but here the input domains are mostly distinct lengths or patterns.
# Pynini's union and optimize will handle it.

################CODE STARTING FROM HERE#####################################
# Using the union property combine all the FSTs that we created so far to create an FST that accepts any string accepted by at least one of its input FSTs
fst_00_to_99 = pynini.union(
    fst_units_with_leading_zeros,           # Handles "0"-"9"
    fst_teens,           # Handles "10"-"19"
    fst_exact_tens,      # Handles "20", "30", "40"
    fst_compound_tens,    # Handles "21-29", "31-39"
    fst_70_to_99         # handles '70-99'
).optimize()
################CODE ENDS HERE##############################################



In [25]:
print("Testing number normalization from 0 to 45 (expect errors > 40):\n")
for i in range(100): # Test up to 45 to see failures
    num_as_str = str(i)
    normalized_text = apply_fst(num_as_str, fst_00_to_99)
    print(f"{num_as_str:>2s} -> {normalized_text}")

Testing number normalization from 0 to 45 (expect errors > 40):

 0 -> zéro
 1 -> un
 2 -> deux
 3 -> trois
 4 -> quatre
 5 -> cinq
 6 -> six
 7 -> sept
 8 -> huit
 9 -> neuf
10 -> dix
11 -> onze
12 -> douze
13 -> treize
14 -> quatorze
15 -> quinze
16 -> seize
17 -> dix-sept
18 -> dix-huit
19 -> dix-neuf
20 -> vingt
21 -> vingt-et-un
22 -> vingt-deux
23 -> vingt-trois
24 -> vingt-quatre
25 -> vingt-cinq
26 -> vingt-six
27 -> vingt-sept
28 -> vingt-huit
29 -> vingt-neuf
30 -> trente
31 -> trente-et-un
32 -> trente-deux
33 -> trente-trois
34 -> trente-quatre
35 -> trente-cinq
36 -> trente-six
37 -> trente-sept
38 -> trente-huit
39 -> trente-neuf
40 -> quarante
41 -> quarante-et-un
42 -> quarante-deux
43 -> quarante-trois
44 -> quarante-quatre
45 -> quarante-cinq
46 -> quarante-six
47 -> quarante-sept
48 -> quarante-huit
49 -> quarante-neuf
50 -> cinquante
51 -> cinquante-et-un
52 -> cinquante-deux
53 -> cinquante-trois
54 -> cinquante-quatre
55 -> cinquante-cinq
56 -> cinquante-six
57 ->

In [26]:
# FST qui consomme '0' et ne produit rien (0:epsilon)
delete_zero = pynutil.delete("0").optimize()

# FST des unités 1 à 9 (qui prend un seul chiffre en entrée)
# Supposons qu'il s'appelle fst_units_1_to_9
# fst_units_1_to_9 = pynini.union(I_O_FST("1", "un"), I_O_FST("2", "deux"), ...).optimize()

# NOUVELLE CONSTRUCTION des unités pour les centaines (01-09)
# Structure: 0 + [1-9] -> <epsilon> + [un à neuf]
fst_01_to_09 = (delete_zero + fst_units_base).optimize()

# FST qui gère 00 à 99 avec entrée de 2 caractères
fst_double_digit_00_to_99 = pynini.union(
    I_O_FST("00", ""),           # 00 -> <epsilon>
    fst_01_to_09,                # 01 à 09 (e.g., "05" -> "cinq")
    fst_teens,           # Handles "10"-"19"
    fst_exact_tens,      # Handles "20", "30", "40"
    fst_compound_tens,    # Handles "21-29", "31-39"
    fst_70_to_99         # handles '70-99'      # 70 à 99 (e.g., "99" -> "quatre-vingt-dix-neuf")
).optimize()

In [27]:
print("Testing number normalization from 0 to 45 (expect errors > 40):\n")
for i in range(100): # Test up to 45 to see failures
    num_as_str = str(i)
    normalized_text = apply_fst(num_as_str, fst_double_digit_00_to_99)
    print(f"{num_as_str:>2s} -> {normalized_text}")

Testing number normalization from 0 to 45 (expect errors > 40):

 0 -> Error: Operation failed, for input:'0'
 1 -> Error: Operation failed, for input:'1'
 2 -> Error: Operation failed, for input:'2'
 3 -> Error: Operation failed, for input:'3'
 4 -> Error: Operation failed, for input:'4'
 5 -> Error: Operation failed, for input:'5'
 6 -> Error: Operation failed, for input:'6'
 7 -> Error: Operation failed, for input:'7'
 8 -> Error: Operation failed, for input:'8'
 9 -> Error: Operation failed, for input:'9'
10 -> dix
11 -> onze
12 -> douze
13 -> treize
14 -> quatorze
15 -> quinze
16 -> seize
17 -> dix-sept
18 -> dix-huit
19 -> dix-neuf
20 -> vingt
21 -> vingt-et-un
22 -> vingt-deux
23 -> vingt-trois
24 -> vingt-quatre
25 -> vingt-cinq
26 -> vingt-six
27 -> vingt-sept
28 -> vingt-huit
29 -> vingt-neuf
30 -> trente
31 -> trente-et-un
32 -> trente-deux
33 -> trente-trois
34 -> trente-quatre
35 -> trente-cinq
36 -> trente-six
37 -> trente-sept
38 -> trente-huit
39 -> trente-neuf
40 -> qu

In [28]:
# FST qui accepte "00" ou <eps> et produit un 's' si le nombre est une centaine exacte (200, 300, etc.)
# Si l'entrée est "00", on produit le 's' (deux cents)
# Si l'entrée n'est pas "00", on produit <eps> (deux cent un)

# FST qui accepte "00" et produit "s" (pour le pluriel de "cent")
# On utilise pynutil.insert("s") car "00" est déjà consommé par le FST des dizaines/unités (voir section 2).
# Simplification : On va rendre l'insertion de 's' facultative.

# FST qui produit 'cent'
fst_cent = I_O_FST("", "cent").optimize()

# FST pour l'insertion du 's' (pluriel)
# Il est inséré uniquement si la suite du nombre est '00'
fst_pluriel_s = pynini.union(pynutil.insert("s"), I_O_FST("", "")).optimize()

In [29]:
# FST pour les unités des centaines (1 à 9)
units_hundreds_map = {k: v for k, v in digit_map.items() if k != "0"} # 1 à 9
fst_units_hundreds_digit = pynini.union(*[I_O_FST(k, v) for k, v in units_hundreds_map.items()]).optimize()


# === A. 100 à 199 ===
# L'unité "1" doit être supprimée en sortie (100 -> cent)
fst_h_1 = pynutil.delete("1") 

# Chemin 100 : 1 + 00  -->  cent
# Note : Nous insérons "cent" directement pour éviter d'utiliser des FSTs qui segmentent mal.
fst_100_exact = I_O_FST("1", "cent").optimize() + I_O_FST("00", "").optimize() 

# Chemin 101-199 : 1 + (01-99) --> cent + [espace] + [dizaines/unités]
# On utilise fst_00_to_99_str_input qui gère l'entrée sur 2 chiffres
fst_101_to_199 = I_O_FST("1", "cent").optimize() + fst_insert_space + fst_double_digit_00_to_99.optimize()

fst_100_to_199_corrected = pynini.union(fst_100_exact, fst_101_to_199).optimize()

In [30]:
apply_fst('100', fst_100_to_199_corrected)

'cent'

In [31]:
# === B. 200 à 999 ===
# Pluriel conditionnel du mot "cent" :
# 1. 200, 300, ... 900 : [2-9] + cent + 's' + 00
# 2. 201, 350, ... 999 : [2-9] + cent + [espace] + [01-99] (pas de 's')

# Unités pour les centaines (2 à 9)
fst_units_2_to_9_digit = pynini.union(*[I_O_FST(k, v) for k, v in units_hundreds_map.items() if k != "1"]).optimize()

# 1. Centaines Exactes (200, 300, ..., 900)
# Structure: [2-9] + " cents" + 00
fst_exact_hundreds = (fst_units_2_to_9_digit + fst_insert_space +I_O_FST("", "cents")).optimize() + I_O_FST("00", "").optimize()


# 2. Centaines Composées (201, 350, ..., 999)
# Structure: [2-9] + " cent " + [01-99]
fst_composed_hundreds = (fst_units_2_to_9_digit + fst_insert_space +I_O_FST("", "cent")).optimize() +fst_insert_space+fst_double_digit_00_to_99.optimize()


# Union 200-999
fst_200_to_999_corrected = pynini.union(fst_exact_hundreds, fst_composed_hundreds).optimize()


# === FST Final Corrigé ===
fst_100_to_999_corrected = pynini.union(fst_100_to_199_corrected, fst_200_to_999_corrected).optimize()

In [32]:
apply_fst("999", fst_composed_hundreds)

'neuf-cent-quatre-vingt-dix-neuf'

In [33]:
# fst_100_to_999 = pynini.union(fst_100_to_199_corrected, fst_100_to_999_corrected).optimize()

In [34]:
# FST pour 1000
fst_1000 = I_O_FST("1000", "mille").optimize()

In [35]:
# Cell 8: Combine all FSTs into a single Normalizer FST

# The order of union can sometimes matter if there are ambiguities,
# but here the input domains are mostly distinct lengths or patterns.
# Pynini's union and optimize will handle it.

################CODE STARTING FROM HERE#####################################
# Using the union property combine all the FSTs that we created so far to create an FST that accepts any string accepted by at least one of its input FSTs
fst_00_to_1000 = pynini.union(
    fst_units_with_leading_zeros,           # Handles "0"-"9"
    fst_teens,           # Handles "10"-"19"
    fst_exact_tens,      # Handles "20", "30", "40"
    fst_compound_tens,    # Handles "21-29", "31-39"
    fst_70_to_99,         # handles '70-99'
    fst_100_to_999_corrected,
    fst_1000
).optimize()
################CODE ENDS HERE##############################################



In [36]:
print("Testing number normalization from 0 to 45 (expect errors > 40):\n")
for i in range(1000): # Test up to 45 to see failures
    num_as_str = str(i)
    normalized_text = apply_fst(num_as_str, fst_00_to_1000)
    print(f"{num_as_str:>2s} -> {normalized_text}")

Testing number normalization from 0 to 45 (expect errors > 40):

 0 -> zéro
 1 -> un
 2 -> deux
 3 -> trois
 4 -> quatre
 5 -> cinq
 6 -> six
 7 -> sept
 8 -> huit
 9 -> neuf
10 -> dix
11 -> onze
12 -> douze
13 -> treize
14 -> quatorze
15 -> quinze
16 -> seize
17 -> dix-sept
18 -> dix-huit
19 -> dix-neuf
20 -> vingt
21 -> vingt-et-un
22 -> vingt-deux
23 -> vingt-trois
24 -> vingt-quatre
25 -> vingt-cinq
26 -> vingt-six
27 -> vingt-sept
28 -> vingt-huit
29 -> vingt-neuf
30 -> trente
31 -> trente-et-un
32 -> trente-deux
33 -> trente-trois
34 -> trente-quatre
35 -> trente-cinq
36 -> trente-six
37 -> trente-sept
38 -> trente-huit
39 -> trente-neuf
40 -> quarante
41 -> quarante-et-un
42 -> quarante-deux
43 -> quarante-trois
44 -> quarante-quatre
45 -> quarante-cinq
46 -> quarante-six
47 -> quarante-sept
48 -> quarante-huit
49 -> quarante-neuf
50 -> cinquante
51 -> cinquante-et-un
52 -> cinquante-deux
53 -> cinquante-trois
54 -> cinquante-quatre
55 -> cinquante-cinq
56 -> cinquante-six
57 ->

In [37]:
import re
from pynini.lib import rewrite
import pynini # Assurez-vous que tous vos FSTs sont définis et disponibles ici

# *******************************************************************
# ** PRÉ-REQUIS : Assurez-vous que 'fst_cardinal_0_to_1000' est défini **
# ** et est votre FST global unissant 0 à 1000. **
# *******************************************************************

def normalize_cardinals_in_sentence(sentence: str, cardinal_fst: pynini.Fst) -> str:
    """
    Parcourt une phrase, identifie les séquences de chiffres consécutifs
    et les remplace par leur équivalent textuel en utilisant un FST.

    Args:
        sentence (str): La phrase contenant des nombres à convertir.
        cardinal_fst (pynini.Fst): Le FST de conversion des nombres cardinaux.

    Returns:
        str: La phrase avec les nombres convertis en texte.
    """
    # 1. Définir le pattern Regex pour trouver les nombres
    # \b : limite de mot (pour éviter de convertir des parties de mots/codes)
    # \d+ : une ou plusieurs occurrences de chiffres (0-9)
    number_pattern = r'\b\d{1,4}\b' # Limiter à 4 chiffres (pour 0 à 1000)

    def replace_match(match: re.Match) -> str:
        """
        Fonction de remplacement appelée par re.sub.
        Elle prend la chaîne numérique trouvée et applique le FST.
        """
        number_str = match.group(0) # Le nombre en chiffres (ex: "3", "201", "1000")

        # ⚠️ Vérification cruciale : le FST ne gère que les nombres jusqu'à 1000.
        # Nous faisons une conversion de sécurité ici.
        if len(number_str) > 4:
            # Si le nombre est trop grand, le laisser en chiffres.
            return number_str
        
        # 2. Appliquer le FST pour la conversion
        try:
            # one_top_rewrite renvoie la meilleure (première) sortie du FST
            converted_text = apply_fst(number_str, cardinal_fst)
            return converted_text
        except:
            # En cas d'échec de la conversion (nombre hors limite FST, ex: "1001" si le FST s'arrête à 1000)
            # Ou si le FST n'a pas de chemin, on retourne la chaîne de chiffres originale.
            print(f"Avertissement : Impossible de convertir '{number_str}' via le FST. Retour au format numérique.")
            return number_str

    # 3. Appliquer la substitution sur toute la phrase
    # re.sub parcourt la phrase et appelle replace_match pour chaque segment correspondant.
    result_sentence = re.sub(number_pattern, replace_match, sentence)

    return result_sentence

# *******************************************************************
# Exemple d'utilisation (Nécessite que vous ayez défini fst_cardinal_0_to_1000)
# *******************************************************************

phrase_test = "J'ai 3 chiens. Le prix est de 200 euros. Il en restait 71."
phrase_normalisee = normalize_cardinals_in_sentence(phrase_test, fst_00_to_1000)
print(phrase_normalisee)
# Sortie attendue : "J'ai trois chiens. Le prix est de deux cents euros. Il en restait soixante-et-onze."

J'ai trois chiens. Le prix est de deux-cents euros. Il en restait soixante-et-onze.


In [38]:
!pip install Levenshtein

In [39]:
import re
import numpy as np


def tokenize(text):
    """
    Tokenisation robuste :
    - Sépare les apostrophes (l'homme → l' homme)
    - Conserve les mots avec apostrophe comme deux tokens
    - Normalise les espaces
    """
    # Sépare toute apostrophe du mot suivant
    text = re.sub(r"(\w)'(\w)", r"\1' \2", text)
    text = re.sub(r"(\W)", r" \1 ", text)     # sépare la ponctuation
    return [t for t in text.split() if t.strip()]


def levenshtein(seq1, seq2):
    """
    Distance de Levenshtein entre deux listes de tokens
    """
    n, m = len(seq1), len(seq2)
    dp = np.zeros((n + 1, m + 1), dtype=int)

    for i in range(n + 1):
        dp[i][0] = i
    for j in range(m + 1):
        dp[0][j] = j

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            cost = 0 if seq1[i - 1] == seq2[j - 1] else 1
            dp[i][j] = min(
                dp[i - 1][j] + 1,       # suppression
                dp[i][j - 1] + 1,       # insertion
                dp[i - 1][j - 1] + cost # substitution
            )
    return dp[n][m]


def wer(reference, hypothesis):
    ref_tokens = tokenize(reference)
    hyp_tokens = tokenize(hypothesis)

    distance = levenshtein(ref_tokens, hyp_tokens)
    return distance / max(len(ref_tokens), 1)


# Exemple
réf = "J'ai trois chiens"
hyp = normalize_cardinals_in_sentence("J'ai 3 chiens", fst_00_to_1000)
print("WER :", wer(réf, hyp))


WER : 0.0


In [40]:
df = pd.read_csv("data/dataset_normalisation_0_1000.csv")

In [41]:
df.iloc[50 : 100]

,input,reference
50,250,deux-cent-cinquante
51,300,trois-cents
52,301,trois-cent-un
53,350,trois-cent-cinquante
54,400,quatre-cents
55,450,quatre-cent-cinquante
56,500,cinq-cents
57,501,cinq-cent-un
58,600,six-cents
59,700,sept-cents


In [42]:
hyp = []
ref = df.reference.to_list()
inpt = df.input.to_list()

for i in inpt:
    hyp.append(normalize_cardinals_in_sentence(str(i), fst_00_to_1000))


In [43]:
wers = []
if len(ref) == len(hyp):
    for i in range(len(ref)):
        wers.append(wer(ref[i], hyp[i]))

print("Wer score = ", sum(wers) / len(wers))

Wer score =  0.004997613080319846


In [44]:
df["hyp"] = hyp

In [45]:
df.iloc[50:100]

,input,reference,hyp
50,250,deux-cent-cinquante,deux-cent-cinquante
51,300,trois-cents,trois-cents
52,301,trois-cent-un,trois-cent-un
53,350,trois-cent-cinquante,trois-cent-cinquante
54,400,quatre-cents,quatre-cents
55,450,quatre-cent-cinquante,quatre-cent-cinquante
56,500,cinq-cents,cinq-cents
57,501,cinq-cent-un,cinq-cent-un
58,600,six-cents,six-cents
59,700,sept-cents,sept-cents


In [49]:
def create_far_archive(output_path="cardinal_numbers.far"):
    """
    Crée un fichier FAR contenant le FST de normalisation
    """
    
    # print("Construction du FST cardinal...")
    # cardinal_fst = build_cardinal_fst()
    
    print(f"Création du fichier FAR: {output_path}")
    
    # Écrire le fichier FAR avec pynini.Far en mode écriture
    far_writer = pynini.Far(output_path, mode="w")
    
    # Ajouter le FST avec la méthode add()
    far_writer.add("CARDINAL", fst_00_to_1000)
    print(f"  ✓ FST 'CARDINAL' ajouté au FAR")
    
    # Fermer le writer
    far_writer.close()
    
    print(f"✓ Fichier FAR créé avec succès: {output_path}")
    return output_path

In [50]:
create_far_archive()

Création du fichier FAR: cardinal_numbers.far
  ✓ FST 'CARDINAL' ajouté au FAR
✓ Fichier FAR créé avec succès: cardinal_numbers.far


'cardinal_numbers.far'

In [53]:
#============================================
# 4. FONCTION DE TEST
# ============================================

def test_fst_from_far(far_path="cardinal_numbers.far"):
    """
    Teste le FST depuis le fichier FAR
    """
    print("\n" + "="*50)
    print("TEST DU FST DEPUIS LE FICHIER FAR")
    print("="*50)
    
    # Lire le fichier FAR
    far_reader = pynini.Far(far_path, mode="r")
    
    # Récupérer le FST
    cardinal_fst = far_reader["CARDINAL"]
    
    # Tests
    test_cases = ["0", "7", "15", "42", "99", "100", "256", "1000"]
    
    for number in test_cases:
        try:
            result = apply_fst(number, cardinal_fst)
            print(f"{number:>4} → {result}")
        except Exception as e:
            print(f"{number:>4} → ERREUR: {e}")
    
    # Fermer le reader
    far_reader.close()

# ============================================
# 5. FONCTION PRINCIPALE
# ============================================

def main():
    """Point d'entrée principal"""
    
    # Créer le fichier FAR
    far_path = create_far_archive("cardinal_numbers.far")
    
    # Tester le FST
    test_fst_from_far(far_path)
    
    print("\n✓ Processus terminé avec succès!")

In [54]:
main()

Création du fichier FAR: cardinal_numbers.far
  ✓ FST 'CARDINAL' ajouté au FAR
✓ Fichier FAR créé avec succès: cardinal_numbers.far

TEST DU FST DEPUIS LE FICHIER FAR
   0 → zéro
   7 → sept
  15 → quinze
  42 → quarante-deux
  99 → quatre-vingt-dix-neuf
 100 → cent
 256 → deux-cent-cinquante-six
1000 → mille

✓ Processus terminé avec succès!
